In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# %pdb on

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# NPBのEPの初期変数
train_ep_process :list[int] = [4, 8, 16, 32, 64]
train_ep_size :list[str] = ["S", "W", "A", "B", "C"]
test_ep_process :list[int] = [128, 256, 512]
test_ep_size :list[str] = ["D", "E", "F"]

# NPBのFTの初期変数
train_ft_process :list[int] = [2, 4, 8, 16, 32, 64]
train_ft_grid_size :list[int] = [32, 64, 128, 256, 512]
train_ft_nit :list[int] = [5, 10, 15, 20, 25]

test_ft_process :list[int] = [128, 256, 512]
test_ft_grid_size :list[int] = [1024, 2048, 4096]
test_ft_nit :list[int] = [30, 40, 50]

# NPBのISの初期変数
train_is_process :list[int] = [2, 4, 8, 16, 32, 64]
train_is_no_of_keys :list[int] = [18, 20, 22, 24, 26, 28, 30, 32]
train_is_max_value :list[int] = [5, 7, 9, 11, 13, 15, 17]

# NPBのMGの初期変数
mg_size :list[int] = [32, 64, 128, 256, 512]
mg_nit: list[int] = [4, 10, 20, 35, 50]

train_mg_process :list[int] = [4, 8, 16, 32, 64]
train_mg_size :list[int] = [4, 8, 16, 32, 64]
train_mg_nit :list[int] = [5, 10, 15, 20, 25]

test_mg_process :list[int] = [128, 256, 512]
test_mg_size :list[int] = [128, 256, 512]
test_mg_nit :list[int] = [30, 40, 50]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

list_csvDir_mg = [
    "./csv_files/mg_1st/",
    "./csv_files/mg_2nd/",
    "./csv_files/mg_3rd/",
]

list_csvDir_ft :list[str] = [
    "./csv_files/ft_1st/",
    "./csv_files/ft_2nd/",
    "./csv_files/ft_3rd/",
]

list_csvDir_ep :list[str] = [
    "./csv_files/ep_1st/",
    "./csv_files/ep_2nd/",
    "./csv_files/ep_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


In [2]:
def return_rawDF_ep(
    list_process: list[int],
    list_size: list[int],
    csvDir: str,
) -> pd.DataFrame:
    """return_rawDF_ep()

    ベンチマークプログラムEPの手動で変更した初期変数におけるプロファイルを取得する関数

    Args:
        list_process(list[int]):プロセス数のリスト
        list_size(list[int]):初期変数sizeのリスト
        csvDir(str):CSVファイルの保持されているディレクトリ

    Returns:
        pd.DataFrame

    """

    list_before_concat_DF: list[pd.DataFrame] = []

    for elem_process in list_process:
        for elem_size in list_size:
            filePath: str = f"{csvDir}ft_grid_size{elem_size}_process{elem_process}.csv"
            if os.path.isfile(filePath):
                try:
                    DF_read_raw: pd.DataFrame = pd.read_csv(filePath)
                    DF_read_raw["process"] = elem_process
                    DF_read_raw["size"] = elem_size
                    list_before_concat_DF.append(DF_read_raw)
                except:
                    warnings.warn(f"{filePath} is empty.")
            else:
                warnings.warn(f"{filePath} doesn't exist")
                continue
    return pd.concat(objs=list_before_concat_DF, axis=0)


def ret_averaged_rawDF_ep(
    list_process: list[int],
    list_size: list[int],
    list_csvDir: list[str],
    resVar: str,
):
    """複数のCSVからDFを取得する関数（ベンチマークプログラムEP）

    列Inclusiveおよび列Exclusiveが秒に変換され、InclusivePerCallもしくはExclusivePerCall列が生成される。

    Args:
        list_process(list[int]):プロセス数のリスト
        list_grid_size(list[int]):グリッドサイズのリスト
        list_nit(list[int]):イテレーション数のリスト
        list_csvDir(list[str]):CSVを保持したディレクトリ名のリスト
        resVar(str):説明変数の文字列

    """

    list_DFs_for_return: list[pd.DataFrame] = []
    for elem_process in list_process:
        for elem_size in list_size:
            list_inputDFs_for_averaged: list[pd.DataFrame] = []
            for elem_csvDir in list_csvDir:
                try:
                    _raw_DF: pd.DataFrame = return_rawDF_ep(
                        list_process=[elem_process],
                        list_size=[elem_size],
                        csvDir=elem_csvDir,
                    )

                    if resVar in ["Exclusive", "Inclusive", "#Call", "#Subrs"]:
                        # resVar 列の整形
                        if resVar in ["Exclusive", "Inclusive"]:
                            _tmp_converted = map(
                                convertPprofTime, list(_raw_DF[resVar])
                            )
                            _raw_DF[resVar] = list(_tmp_converted)
                        # {resVar}PerCall 列の生成
                        _raw_DF = add_perCallColumn(
                            inputDF=_raw_DF,
                            divisorColName="#Call",
                            dividendColName=resVar,
                            targetColumnName=f"{resVar}PerCall",
                        )
                except:
                    continue

                list_inputDFs_for_averaged.append(_raw_DF)
            list_DFs_for_return.append(
                ret_averagedDF(inputDFs=list_inputDFs_for_averaged, resVar=resVar)
            )
    return pd.concat(objs=list_DFs_for_return, axis=0)

In [3]:
_tmp_DF :pd.DataFrame = ret_averaged_rawDF_ft(list_process=train_ft_process, list_grid_size=train_ft_grid_size, list_nit=train_ft_nit, list_csvDir=list_csvDir_ft, resVar="Exclusive")
print(_tmp_DF.head())

_tmp_DF :pd.DataFrame = ret_averaged_rawDF_ft(list_process=test_ft_process, list_grid_size=test_ft_grid_size, list_nit=test_ft_nit, list_csvDir=list_csvDir_ft, resVar="Exclusive")
print(_tmp_DF.head())

_tmp_DF :pd.DataFrame = ret_averaged_rawDF_mg(list_process=train_mg_process, list_size=train_mg_size, list_nit=train_mg_nit, list_csvDir=list_csvDir_mg, resVar="Exclusive")
print(_tmp_DF.head())

_tmp_DF :pd.DataFrame = ret_averaged_rawDF_mg(list_process=test_mg_process, list_size=test_mg_size, list_nit=test_mg_nit, list_csvDir=list_csvDir_mg, resVar="Exclusive")
print(_tmp_DF.head())

trainDF_EP :pd.DataFrame = ret_averaged_rawDF_ep(list_process=train_ep_process, list_size=train_ep_size, list_csvDir=list_csvDir_ep, resVar="Exclusive")
print(trainDF_EP.head())

trainDF_EP :pd.DataFrame = ret_averaged_rawDF_ep(list_process=test_ep_process, list_size=test_ep_size, list_csvDir=list_csvDir_ep, resVar="Exclusive")
print(trainDF_EP.head())

   %Time  Exclusive Inclusive  #Call  #Subrs              Name  process  \
0  100.0   0.000004     6,054    1.0     1.0  .TAU_application        2   
1  100.0   0.000088     6,054    1.0    65.0                FT        2   
2   90.8   7.144667     5,497    1.0     0.0    MPI_Finalize()        2   
3    9.1   0.000169       549    1.0    11.0             SETUP        2   
4    9.1   0.511000       548    1.0     0.0        MPI_Init()        2   

   grid_size  nit  ExclusivePerCall  
0         32    5          0.000004  
1         32    5          0.000078  
2         32    5          5.497000  
3         32    5          0.000172  
4         32    5          0.548000  
   %Time  Exclusive Inclusive    #Call     #Subrs              Name  process  \
0  100.0   0.000004     5,661      1.0      1.000  .TAU_application      128   
1  100.0   0.000337     5,661      1.0    124.016                FT      128   
2   70.4   0.000165     3,984     27.0    108.000               FFT      128   
3

/tmp/ipykernel_17048/1906563771.py:34: UserWarning: ./csv_files/ep_1st/ft_grid_sizeS_process4.csv doesn't exist
  warnings.warn(f"{filePath} doesn't exist")
/tmp/ipykernel_17048/1906563771.py:34: UserWarning: ./csv_files/ep_2nd/ft_grid_sizeS_process4.csv doesn't exist
  warnings.warn(f"{filePath} doesn't exist")
/tmp/ipykernel_17048/1906563771.py:34: UserWarning: ./csv_files/ep_3rd/ft_grid_sizeS_process4.csv doesn't exist
  warnings.warn(f"{filePath} doesn't exist")


NameError: name 'warning' is not defined

# 必要な結果

* 関数コール回数予測
    * モデル適合度
    * モデル予測精度
* 実行時間予測
    * モデル適合度
    * モデル予測精度

# 必要なブツ

* モデル構築用データ
* 予測対象用データ
* 関数コール回数の予測モデル
* 関数の総実行時間の予測モデル
* 1コール当たりの関数の総実行時間の予測モデル


In [ ]:
class ContentsForExtraP:
    """class ContentsForExtraP

    ExtraPによるモデルの生成

    生成されるモデルは次の通り。

    * 関数コール回数の予測モデル
    * 関数の総実行時間の予測モデル
    * 1コール当たりの関数の総実行時間の予測モデル

    """

    def __init__(
        self,
        trainDF: pd.DataFrame,
        testDF: pd.DataFrame,
        expVars: list[str],
        resVars: list[str],
        resVarsPerCall: list[str],
    ):
        """__init__()

        初期化変数

        Args:
            trainDF(pd.DataFrame):モデル構築用データ
            testDF(pd.DataFrame):予測対象用データ
            expVars(list[str]):説明変数を格納したリスト
            resVars(list[str]):目的変数を格納したリスト
            resVarsPerCall(list[str]):1コール当たりの目的変数を格納したリスト

        """
        self.trainDF = trainDF
        self.testDF = testDF
        self.expVars = expVars
        self.resVars = resVars
        self.resVarsPerCall = resVarsPerCall

    def _build_model(self, resVar: str):
        """_build_model()

        引数から指定した条件でモデルを構築する。

        Args:
            resVar(str):目的変数

        """

        pass

    def build_all_models(self):
        """build_all_models()

        モデルを構築する関数。目的変数はself.resVars, self.resVarsPerCallを利用。

        """

        pass

    def return_models(self):
        """return_models()

        構築した各種モデルを返す関数。

        """

        pass